In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()
spark.version

'3.0.0'

In [2]:
%env SPARK_VERSION=3.0.0# TODO PUT YOUR VALUE

env: SPARK_VERSION=3.0.0# TODO PUT YOUR VALUE


In [3]:
pip install pydeequ

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pretty_html_table

Note: you may need to restart the kernel to use updated packages.


In [3]:
# TODO create spark session with jdbc driver path
from pyspark.sql import SparkSession, Row
import pydeequ

spark = (SparkSession\
    .builder\
    .master("local[*]")\
    .appName("Connection through JDBC")\
    .config("spark.driver.extraClassPath","/home/jovyan/work/sqljdbc42.jar") \
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)\
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())

In [4]:
# TODO connect to DB with Spark using JDBC connection to read the data
#server_url = "jdbc:sqlserver://192.168.1.138:1433" # think what should be here
server_url = "jdbc:sqlserver://192.168.100.94:1433" # think what should be here
database_name = "TRN_2"
url = server_url + ";" + "databaseName=" + database_name + ";"



table = "hr.employees"
user = "AVainilovich" # your user creds here
password = "1411_Omg" # your user creds here




df=spark.read \
    .format("jdbc") \
    .option("driver" , "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
    .option("url",f"jdbc:sqlserver://192.168.100.94:1433;databaseName={database_name}") \
    .option("dbtable", table) \
    .option("user", user) \
    .option("password", password) \
    .load()

df.show(20)

+-----------+-----------+----------+--------------------+------------+----------+------+--------+----------+-------------+
|employee_id| first_name| last_name|               email|phone_number| hire_date|job_id|  salary|manager_id|department_id|
+-----------+-----------+----------+--------------------+------------+----------+------+--------+----------+-------------+
|        100|     Steven|      King|steven.king@sqltu...|515.123.4567|1987-06-17|     4|24000.00|      null|            9|
|        101|      Neena|   Kochhar|neena.kochhar@sql...|515.123.4568|1989-09-21|     5|17000.00|       100|            9|
|        102|        Lex|   De Haan|lex.de haan@sqltu...|515.123.4569|1993-01-13|     5|17000.00|       100|            9|
|        103|  Alexander|    Hunold|alexander.hunold@...|590.423.4567|1990-01-03|     9| 9000.00|       102|            6|
|        104|      Bruce|     Ernst|bruce.ernst@sqltu...|590.423.4568|1991-05-21|     9| 6000.00|       103|            6|
|        105|   

In [5]:
### Data Analyzers section
# TODO analyze data here

from pydeequ.analyzers import *

analysisResult = AnalysisRunner(spark) \
                    .onData(df) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("employee_id")) \
                    .addAnalyzer(CountDistinct("phone_number")) \
                    .addAnalyzer(Mean("salary")) \
                    .addAnalyzer(Minimum("salary")) \
                    .addAnalyzer(Maximum("salary")) \
                    .run()

analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+------------+-------------+-------+
| entity|    instance|         name|  value|
+-------+------------+-------------+-------+
| Column| employee_id| Completeness|    1.0|
|Dataset|           *|         Size|   40.0|
| Column|phone_number|CountDistinct|   34.0|
| Column|      salary|         Mean| 8060.0|
| Column|      salary|      Minimum| 2500.0|
| Column|      salary|      Maximum|24000.0|
+-------+------------+-------------+-------+



In [6]:
### Data profiling section
# TODO profile data here

from pydeequ.profiles import *

resultProfile = ColumnProfilerRunner(spark) \
    .onData(df) \
    .run()

for col, profile in resultProfile.profiles.items():
    print(profile)


StandardProfiles for column: first_name: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 36,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 40
    },
    "histogram": [
        [
            "Den",
            1,
            0.025
        ],
        [
            "John",
            2,
            0.05
        ],
        [
            "Karen",
            2,
            0.05
        ],
        [
            "William",
            1,
            0.025
        ],
        [
            "Alexander",
            2,
            0.05
        ],
        [
            "Pat",
            1,
            0.025
        ],
        [
            "Lex",
            1,
            0.025
        ],
        [
            "Nancy",
            1,
            0.025
        ],
        [
            "Shelli",
            1,
            0.025
        ],
  

In [7]:
### Constraint Suggestions section
# TODO find meaningful constraints here

from pydeequ.suggestions import *

suggestionResult = ConstraintSuggestionRunner(spark) \
                   .onData(df) \
                   .addConstraintRule(DEFAULT()) \
                   .run()

# Constraint Suggestions in JSON format
print(json.dumps(suggestionResult, indent = 2))

{
  "constraint_suggestions": [
    {
      "constraint_name": "CompletenessConstraint(Completeness(first_name,None))",
      "column_name": "first_name",
      "current_value": "Completeness: 1.0",
      "description": "'first_name' is not null",
      "suggesting_rule": "CompleteIfCompleteRule()",
      "rule_description": "If a column is complete in the sample, we suggest a NOT NULL constraint",
      "code_for_constraint": ".isComplete(\"first_name\")"
    },
    {
      "constraint_name": "CompletenessConstraint(Completeness(employee_id,None))",
      "column_name": "employee_id",
      "current_value": "Completeness: 1.0",
      "description": "'employee_id' is not null",
      "suggesting_rule": "CompleteIfCompleteRule()",
      "rule_description": "If a column is complete in the sample, we suggest a NOT NULL constraint",
      "code_for_constraint": ".isComplete(\"employee_id\")"
    },
    {
      "constraint_name": "ComplianceConstraint(Compliance('employee_id' has no negativ

In [8]:
### Constraint Verification section
# TODO check selected constraints here and make beautify the report

import pandas as pd
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Employees Review Check")
checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck( check.hasSize(lambda x: x >= 40) \
    .hasMin("salary", lambda x: x >= 2500) \
    .hasMax("salary", lambda x: x >= 24000) \
    .isComplete("manager_id") \
    .isUnique("phone_number") \
    .isComplete("hire_date") \
    .isContainedIn("phone_number", ["515.123.4567", "515.123.4568", "515.123.4569"])) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

Python Callback server started!
+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|               check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+--------------------+-----------+------------+--------------------+-----------------+--------------------+
|Employees Review ...|    Warning|     Warning|SizeConstraint(Si...|          Success|                    |
|Employees Review ...|    Warning|     Warning|MinimumConstraint...|          Success|                    |
|Employees Review ...|    Warning|     Warning|MaximumConstraint...|          Success|                    |
|Employees Review ...|    Warning|     Warning|CompletenessConst...|          Failure|Value: 0.975 does...|
|Employees Review ...|    Warning|     Warning|UniquenessConstra...|          Success|                    |
|Employees Review ...|    Warning|     Warning|CompletenessConst...|          Success|                  

In [ ]:
### Make as beautiful test report as you can for constraints verification step (csv, html, etc.)

import pandas
import pretty_html_table

#get checks results in json format
checkResult_json = VerificationResult.checkResultsAsJson(spark, checkResult)

#create pandas dataframe from verification's checkResult
df = pandas.DataFrame(checkResult_json)

#declare html template
html_string = '''
<html>
  <head><title>HTML Pandas Dataframe with CSS</title></head>
  <link rel="stylesheet" type="text/css" href="df_style.css"/>
  <body>
    {table}
  </body>
</html>
'''

#render dataframe as html
#html_string.format(table=df.to_html(classes='mystyle'))
#result_report = checkResult_df.toPandas().to_html()
html = html_string.format(table=df.to_html(classes='mystyle'))
#print(result_report)

#write html to file
text_file = open("result_report.html", "w")
text_file.write(html)
text_file.close()